In [1]:
import os
import sys
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.image as mpimg
import optuna
import joblib
import math
import ast
import logging
from pathlib import Path  
from msig import Motif, NullModel

optuna.logging.set_verbosity(optuna.logging.WARNING)

# Load YAML configuration
config_path = "config.yaml" 
with open(config_path, "r") as file:
    config = yaml.safe_load(file)

BASE_DIR = Path(config["base_dir"]).resolve()

RESULTS_DIR = BASE_DIR / config["results_dir"]
IMAGES_DIR = BASE_DIR / config["images_dir"]
DATA_DIR = BASE_DIR / config["data_dir"]
DATASET_PATH = BASE_DIR / config["dataset_path"]
RESULTS_MOTIF_DIR = BASE_DIR / config["results_motif_dir"]

VARIABLES = config["variables"]
NORMALIZE_FLAGS = config["normalize_flags"]
STUMPY_EXCL_ZONE_DENOM = config["stumpy_excl_zone_denom"]
TOP_K_MP = config["top_k_mp"]
INCLUDE = config["include"]
NORMALIZE = config["normalize"]
SUBSEQUENCES_LENGTHS = config["subsequences_lengths"]
NTOP_MOTIFS = config["ntop_motifs"]
MOTIF_SIZE = config["motif_size"]
LOOKBACK_PERIOD = config["lookback_period"]
STEP = config["step"]
FORECAST_PERIOD = config["forecast_period"]

print(f"Results will be saved in: {RESULTS_DIR}")
print(f"Images will be saved in: {IMAGES_DIR}")
print(f"Data will be accessed from: {DATA_DIR}")
print(f"Dataset path: {DATASET_PATH}")

if '__file__' in globals():
    # For standalone scripts
    base_dir = Path(__file__).parent.resolve()
else:
    # For Jupyter or interactive environments
    base_dir = Path(os.getcwd()).resolve()

sys.path.append(str(base_dir / "../"))


Results will be saved in: /home/mgsilva/motifpred/results/household
Images will be saved in: /home/mgsilva/motifpred/images/household
Data will be accessed from: /home/mgsilva/motifpred/data/household
Dataset path: /home/mgsilva/motifpred/data/household/data_5min_resampled_globalactive_reactive.csv


In [2]:
mp_stats_table = pd.read_csv(os.path.join(RESULTS_DIR, f"mp_stats_table_normalized_{NORMALIZE}_top_{TOP_K_MP}.csv"))
mp_stats_table = mp_stats_table[mp_stats_table["m"] == MOTIF_SIZE]
top_motifs = mp_stats_table.sort_values(by=["#Matches", "ID"], ascending=[False, True]).head(NTOP_MOTIFS)
top_motifs = top_motifs[["m", "Indices"]]

In [3]:
# Import shared setup
from utils.setup import seed, device, early_stopper, pipeline, test_tensor

# Example usage
print(f"Device: {device}")
test_tensor()

Device: cuda
tensor([[0.3126, 0.3791, 0.3087],
        [0.0736, 0.4216, 0.0691],
        [0.2332, 0.4047, 0.2162],
        [0.9927, 0.4128, 0.5938],
        [0.6128, 0.1519, 0.0453]])


In [4]:
#load data as 
data_df = pd.read_csv(DATASET_PATH, index_col=0).astype(float)
data_df = data_df[VARIABLES]
data = data_df.values.T
data

array([[1.2944, 1.3644, 1.4672, ..., 0.7808, 0.7676, 0.634 ],
       [0.    , 0.0776, 0.2188, ..., 0.1644, 0.1492, 0.0608]])

In [5]:
import os
import numpy as np
import joblib
import ast
import pandas as pd
from utils.utils import create_dataset, get_best_model_results_traindevtest, plot_best_model_results_traindevtest, plot_preds_vs_truevalues
from utils.train_pipeline import get_preds_best_config_train_val_test
from models.ffnn_pytorch import FFNN
from models.lstm_pytorch import LSTM
from models.cnn_pytorch import CNN
from models.tcn_pytorch import TCN
from models.transformer_pytorch import Transformer
from models.baseline_pytorch import BaselineAverage, BaselineLastDifference


models = [ "FFNN", "LSTM", "CNN", "TCN", "Transformer", "Baseline"]
inputs = ["Series", "Series_Masking", "Indexes"]

n_trials = 100
num_epochs = 500

def process_baseline_model(model_class, input_name, X, normalize_flags, n_trials, num_epochs, seed, pipeline, y, motif_id):
    """Process baseline models."""
    model_name = f"{model_class.__name__}{input_name}"
    model_results_dir = os.path.join(RESULTS_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs_motif_{motif_id}")
    os.makedirs(model_results_dir, exist_ok=True)
    
    if os.path.exists(os.path.join(model_results_dir, "best_model_results.csv")):
        print(f"Model {model_name} already exists for motif {motif_id}")
        return
    
    print(f"Processing model {model_name} for motif {motif_id}")
    study = joblib.load(os.path.join(model_results_dir, "study.pkl"))
    train_losses, val_losses, best_epoch, test_loss, test_mae, test_rmse = get_best_model_results_traindevtest(study)
    
    _, _, _, test_losses, test_mae, test_rmse, all_predictions, all_true_values = get_preds_best_config_train_val_test(
        study, pipeline, model_class, "Baseline", [], num_epochs=num_epochs, seed=seed, X=X, y=y, normalize_flags=normalize_flags
    )

    if not np.allclose(test_loss, test_losses, atol=0.1):
        print(f"Test loss: {test_loss}")
        print(f"Retrained test loss: {test_losses}")
        raise Exception("Best model test loss does not match the one obtained from the study")

    
    results = pd.DataFrame({
        "test_loss": [test_loss],
        "test_mae": [test_mae],
        "test_rmse": [test_rmse]
    })
    results.to_csv(os.path.join(model_results_dir, "best_model_results.csv"), index=False)

def process_non_baseline_model(model_type, model_params_keys, input_name, X, normalize_flags, num_epochs, seed, pipeline, y, motif_id):
    """Process non-baseline models."""
    model_name = f"{model_type}{input_name}"
    model_results_dir = os.path.join(RESULTS_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs_motif_{motif_id}")
    os.makedirs(model_results_dir, exist_ok=True)
    
    if os.path.exists(os.path.join(model_results_dir, "best_model_results.csv")):
        print(f"Model {model_name} already exists for motif {motif_id}")
        return
    
    study = joblib.load(os.path.join(model_results_dir, "study.pkl"))
    train_losses, val_losses, best_epoch, test_loss, test_mae, test_rmse  = get_best_model_results_traindevtest(study)
    
    _, _, _, retrained_test_losses, retrained_test_mae, retrained_test_rmse, retrained_all_predictions, retrained_all_true_values = get_preds_best_config_train_val_test(
        study, pipeline, eval(model_type), model_type, model_params_keys, num_epochs=num_epochs, seed=seed, X=X, y=y, normalize_flags=normalize_flags
    )
    
    if not np.allclose(test_loss, retrained_test_losses, atol=50):
        print(f"Test loss: {test_loss}")
        print(f"Retrained test loss: {retrained_test_losses}")
        raise Exception("Best model test loss does not match the one obtained from the study")
    
    results = pd.DataFrame({
        "test_loss": [test_loss],
        "test_mae": [test_mae],
        "test_rmse": [test_rmse]
    })
    results.to_csv(os.path.join(model_results_dir, "best_model_results.csv"), index=False)

# Loop through each motif
for i, top_motif in top_motifs.iterrows():
    motif_indexes = sorted(ast.literal_eval(top_motif["Indices"]))
    print(f"Evaluating motif {i+1} with size {MOTIF_SIZE} and {len(motif_indexes)} indexes")
    
    X_series, X_indices, X_mask, y = create_dataset(data, LOOKBACK_PERIOD, STEP, FORECAST_PERIOD, motif_indexes, MOTIF_SIZE)
    
    for model_type in models:
        for input_name in inputs:
            normalize_flags = NORMALIZE_FLAGS
            model_params_map = {
                "FFNN": ["hidden_sizes_list"],
                "LSTM": ["hidden_sizes_list"],
                "CNN": ["kernel_size", "num_filters_list", "pool_size"],
                "TCN": ["kernel_size", "num_channels_list", "dropout"],
                "Transformer": ["d_model", "n_heads", "e_layers", "dim_feedforward", "dropout"]
            }
            
            if model_type == "Baseline":
                if input_name != "Indexes":
                    continue

                X = {"X_series": X_series, "X_mask": X_mask, "X_indices": X_indices}
                normalize_flags = {"X_series": True, "X_mask": False, "X_indices": False}

                for model_class in [BaselineAverage, BaselineLastDifference]:
                    process_baseline_model(model_class, input_name, X, normalize_flags, 1, 1, seed, pipeline, y, i+1)
            else:
                if input_name == "Series":
                    X = {"X_series": X_series}
                elif input_name == "Series_Masking":
                    X = {"X_series": X_series, "X_mask": X_mask}
                else:
                    X = {"X_indices": X_indices}


                model_params_map = {
                    "FFNN": ["hidden_sizes_list"],
                    "LSTM": ["hidden_sizes_list"],
                    "CNN": ["kernel_size", "num_filters_list", "pool_size"],
                    "TCN": ["kernel_size", "num_channels_list", "dropout"],
                    "Transformer": ["d_model", "n_heads", "e_layers", "dim_feedforward", "dropout"]
                }
                print(f"Processing model {model_type}{input_name} for motif {i+1}")
                process_non_baseline_model(
                    model_type, model_params_map[model_type], input_name, X, normalize_flags, num_epochs, seed, pipeline, y, i+1
                )


Evaluating motif 6 with size 24 and 359 indexes
Processing model FFNNSeries for motif 6
Model FFNNSeries already exists for motif 6
Processing model FFNNSeries_Masking for motif 6
Model FFNNSeries_Masking already exists for motif 6
Processing model FFNNIndexes for motif 6
Model FFNNIndexes already exists for motif 6
Processing model LSTMSeries for motif 6
Model LSTMSeries already exists for motif 6
Processing model LSTMSeries_Masking for motif 6
Model LSTMSeries_Masking already exists for motif 6
Processing model LSTMIndexes for motif 6
Model LSTMIndexes already exists for motif 6
Processing model CNNSeries for motif 6
Model CNNSeries already exists for motif 6
Processing model CNNSeries_Masking for motif 6
Model CNNSeries_Masking already exists for motif 6
Processing model CNNIndexes for motif 6
Model CNNIndexes already exists for motif 6
Processing model TCNSeries for motif 6
Model TCNSeries already exists for motif 6
Processing model TCNSeries_Masking for motif 6
Model TCNSeries_Mas

Exception: Best model test loss does not match the one obtained from the study

In [ ]:
import numpy as np
from scipy import stats
rng = np.random.default_rng()
from scipy.stats import ttest_rel


models = [ "FFNN", "LSTM", "CNN", "TCN", "Transformer" "Baseline"]
inputs = ["Series", "Series_Masking", "Indexes"]


results_df = pd.DataFrame(columns=["model", "input", "motif", "mae", "rmse"])

# Loop through each motif
for i, top_motif in top_motifs.iterrows():
    motif_indexes = sorted(ast.literal_eval(top_motif["Indices"]))
    motif_id = i + 1
    print(f"Processing motif {motif_id} with {len(motif_indexes)} indexes")
    
    for model_type in models:
        for input_name in inputs:
            # Handle baseline-specific logic
            if model_type == "Baseline":
                n_trials, num_epochs = (1, 1)
                if input_name != "Indexes":
                    continue
                
                # Process both BaselineAverage and BaselineLastDifference
                baseline_variants = ["BaselineAverage", "BaselineLastDifference"]
                for baseline_type in baseline_variants:
                    model_name = f"{baseline_type}{input_name}"
                    print(f"Processing Model: {model_name}")

                    # Construct the results directory path
                    model_results_dir = os.path.join(RESULTS_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs_motif_{motif_id}")
                    results_file = os.path.join(model_results_dir, "best_model_results.csv")

                    # Skip if results file doesn't exist
                    if not os.path.exists(results_file):
                        print(f"Results file for {model_name} not found. Skipping.")
                        continue

                    # Load results from CSV
                    results = pd.read_csv(results_file)
                    maes = results["test_mae"].values
                    rmses = results["test_rmse"].values

                    # Add results to the dataframe
                    for i in range(len(maes)):  # Assuming results have folds
                        results_df = pd.concat([
                            results_df,
                            pd.DataFrame([{
                                "model": baseline_type,
                                "input": input_name,
                                "motif": i + 1,
                                "mae": maes[i],
                                "rmse": rmses[i]
                            }])
                        ], ignore_index=True)

            else:
                n_trials = 100
                num_epochs = 500

                model_name = f"{model_type}{input_name}"
                print(f"Processing Model: {model_name}")

                # Construct the results directory path
                model_results_dir = os.path.join(RESULTS_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs_motif_{motif_id}")
                results_file = os.path.join(model_results_dir, "best_model_results.csv")

                # Skip if results file doesn't exist
                if not os.path.exists(results_file):
                    print(f"Results file for {model_name} not found. Skipping.")
                    continue

                # Load results from CSV
                results = pd.read_csv(results_file)
                maes = results["test_mae"].values
                rmses = results["test_rmse"].values

                # Add results to the dataframe
                for i in range(len(maes)):  # Assuming results have folds
                    results_df = pd.concat([
                        results_df,
                        pd.DataFrame([{
                            "model": model_type,
                            "input": input_name,
                            "motif": i + 1,
                            "mae": maes[i],
                            "rmse": rmses[i]
                        }])
                    ], ignore_index=True)

# Display the results
print(results_df)

Processing motif 6 with 359 indexes
Processing Model: FFNNSeries
Processing Model: FFNNSeries_Masking
Processing Model: FFNNIndexes
Processing Model: LSTMSeries
Processing Model: LSTMSeries_Masking
Processing Model: LSTMIndexes
Processing Model: CNNSeries
Processing Model: CNNSeries_Masking
Processing Model: CNNIndexes
Processing Model: TCNSeries
Processing Model: TCNSeries_Masking
Processing Model: TCNIndexes
Processing Model: BaselineAverageIndexes
Processing Model: BaselineLastDifferenceIndexes
Processing motif 18 with 358 indexes
Processing Model: FFNNSeries
Processing Model: FFNNSeries_Masking
Processing Model: FFNNIndexes
Processing Model: LSTMSeries
Processing Model: LSTMSeries_Masking
Processing Model: LSTMIndexes
Processing Model: CNNSeries
Processing Model: CNNSeries_Masking
Processing Model: CNNIndexes
Processing Model: TCNSeries
Processing Model: TCNSeries_Masking
Processing Model: TCNIndexes
Processing Model: BaselineAverageIndexes
Processing Model: BaselineLastDifferenceI

/tmp/ipykernel_280421/393695975.py:83: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([


In [ ]:
#average fold results for each model and input
avg_results_df = results_df.groupby(["model", "input"]).mean().reset_index()
print(avg_results_df)

                     model           input motif        mae       rmse
0          BaselineAverage         Indexes   1.0  37.434675  48.206358
1   BaselineLastDifference         Indexes   1.0  40.447647  53.754295
2                      CNN         Indexes   1.0  31.925813  42.335421
3                      CNN          Series   1.0  32.803692  42.631120
4                      CNN  Series_Masking   1.0  35.722453  45.665712
5                     FFNN         Indexes   1.0  31.844345  42.039991
6                     FFNN          Series   1.0  34.523452  44.126491
7                     FFNN  Series_Masking   1.0  33.268448  45.107064
8                     LSTM         Indexes   1.0  30.800459  40.969523
9                     LSTM          Series   1.0  32.418578  41.870246
10                    LSTM  Series_Masking   1.0  31.152064  40.939259
11                     TCN         Indexes   1.0  31.660012  42.332847
12                     TCN          Series   1.0  32.181380  41.624569
13    

In [ ]:
# Define parameters
models_1 = ["FFNN" ]
input_types_1 = ["Series_Masking","Indexes"]
models_2 = ["BaselineAverage" ]
input_types_2 = ["Indexes"]

# Filter data for the selected input types

results = []
for model1 in models_1:
    for model2 in models_2:
        for input_1 in input_types_1:
            for input_2 in input_types_2:
                for metric in ["mae", "rmse"]:

                    data1 = results_df[(results_df['model'] == model1) & (results_df['input'] == input_1)].sort_values('motif')[metric]
                    data2 = results_df[(results_df['model'] == model2) & (results_df['input'] == input_2)].sort_values('motif')[metric]

                    if len(data1) == len(data2):
                        t_stat, p_value = ttest_rel(data1, data2, alternative='less')
                        results.append({
                            "Model_1": model1,
                            "InputType_1": input_1,
                            "Model_2": model2,
                            "InputType_2": input_2,
                            "Metric": metric,
                            "P-Value": p_value
                        })

# Convert results to DataFrame and display
pval_results_df = pd.DataFrame(results)
pval_results_df

,Model_1,InputType_1,Model_2,InputType_2,Metric,P-Value
0,FFNN,Series_Masking,BaselineAverage,Indexes,mae,0.155132
1,FFNN,Series_Masking,BaselineAverage,Indexes,rmse,0.236713
2,FFNN,Indexes,BaselineAverage,Indexes,mae,0.107958
3,FFNN,Indexes,BaselineAverage,Indexes,rmse,0.088931
